In [ ]:
# In your Jupyter cell

# Standard scientific libraries
import numpy as np
import matplotlib.pyplot as plt

# Libraries for interactive widgets
import ipywidgets as widgets
from IPython.display import display, clear_output

# Import our custom model class from the .py file
from predator_prey_model import PredatorPreyChemostat

# 1. Define the widgets for the parameters we want to control.
delta_slider = widgets.FloatSlider(
    value=0.95,  # A value in the oscillatory regime
    min=0.1,
    max=1.5,
    step=0.01,
    description='Dilution Rate (δ):',
    continuous_update=False, # Important: only updates when we release the slider
    readout_format='.2f'
)

Ni_slider = widgets.FloatSlider(
    value=80.0,
    min=20.0,
    max=500.0,
    step=10.0,
    description='Nitrogen Inflow (Ni):',
    continuous_update=False,
    readout_format='.1f'
)

# MODIFIED: Add a new slider for lambda (senescence rate)
lambda_slider = widgets.FloatSlider(
    value=0.4,  # The value used in the paper
    min=0.0,
    max=1.5,
    step=0.05,
    description='Senescence (λ):',
    continuous_update=False,
    readout_format='.2f'
)


# 2. Creating an Output widget to hold and display the plots.
output_plot = widgets.Output()

# 3. Defining the function that runs the simulation and creates the plot.
# MODIFIED: The function now accepts 'lam' as an argument.
def update_simulation_plot(delta, Ni, lam):
    """Runs simulation and updates the plot within the Output widget."""
    with output_plot:
        # Clears the previous plot first
        clear_output(wait=True)
        
        # MODIFIED: Pass the 'lam' value from the slider when creating the system object.
        system = PredatorPreyChemostat(delta=delta, Ni=Ni, lam=lam)
        
        initial_conditions = [60.0, 10.0, 5.0, 5.0]
        simulation_time = (0, 300) # Increased time to see long-term behavior
        t_eval_points = np.linspace(simulation_time[0], simulation_time[1], 1500)
        
        results = system.run_simulation(initial_conditions, simulation_time, t_eval_points)
        
        # Creating the figure and axes for plotting
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
        
        # Plot 1: Time Series
        ax1.set_title('Population Dynamics Over Time')
        ax1.set_xlabel('Time (days)')
        ax1.set_ylabel('Prey (Chlorella)', color='g')
        ax1.plot(results['time'], results['Chlorella'], color='g')
        ax1.tick_params(axis='y', labelcolor='g')
        ax1.grid(True)
        
        ax1_twin = ax1.twinx()
        ax1_twin.set_ylabel('Predator (Brachionus)', color='k')
        ax1_twin.plot(results['time'], results['Total_Brachionus'], color='k')
        ax1_twin.tick_params(axis='y', labelcolor='k')
        
        # Plot 2: Phase Portrait
        ax2.set_title('Phase Space Portrait')
        ax2.set_xlabel('Prey Population (Chlorella)')
        ax2.set_ylabel('Predator Population (Brachionus)')
        ax2.plot(results['Chlorella'], results['Total_Brachionus'], color='darkblue', lw=1)
        ax2.grid(True)
        
        fig.tight_layout()
        plt.show()

# 4. Link the sliders to the update function.
# MODIFIED: Add the new lambda_slider to the dictionary.
widgets.interactive_output(
    update_simulation_plot, 
    {'delta': delta_slider, 'Ni': Ni_slider, 'lam': lambda_slider}
)

# 5. Display the user interface.
# MODIFIED: Add the new lambda_slider to the VBox of controls.
controls = widgets.VBox([delta_slider, Ni_slider, lambda_slider])
dashboard = widgets.VBox([controls, output_plot])

# Finally, display the complete dashboard
display(dashboard)